In [1]:
import os
import sys
import numpy as np
from PIL import Image
from tqdm import tqdm

### Functions for BundleFusion [dataset](http://graphics.stanford.edu/projects/bundlefusion/)

In [2]:
# rewritting sensorData.h

def make_matrix(data):
    return np.array([np.array(d, dtype=np.float32) for d in data[1:]]).reshape((4, 4))

def read_info(filename):
    header_data = {}
    with open(filename) as file:
        for line in file:
            print(line)
            tmp = line.strip().split('=')
            if 'Intrinsic' in tmp[0] or 'Extrinsic' in tmp[0]:
                header_data[tmp[0].rstrip()] = make_matrix(tmp[-1].rstrip().split(' '))
            else:
                print(tmp[-1])
                header_data[tmp[0].rstrip()] = tmp[-1].strip()
                    
    return header_data
  
def read_image(filename):
    im = Image.open(filename)
    return np.asarray(im)
    
def read_frames_bundlefusion(folder, num_frames):
    frames = {'m_colorCompressed': [], 'm_depthCompressed': [], 'm_cameraToWorld': [],
              'm_timeStampColor': [], 'm_timeStampDepth': [], 
              'm_colorSizeBytes': [], 'm_depthSizeBytes': []}
    frames['num_frames'] = num_frames
    for i in range(int(num_frames)):
        frames['m_colorCompressed'].append(read_image(folder + f'/frame-{i:06}.color.jpg'))
        frames['m_timeStampColor'].append(i)
        frames['m_depthCompressed'].append(read_image(folder + f'/frame-{i:06}.depth.png'))
        frames['m_timeStampDepth'].append(i)
        
        frames['m_cameraToWorld'].append(np.loadtxt(folder + f'/frame-{i:06}.pose.txt'))
        h, w, c = frames['m_colorCompressed'][-1].shape
        frames['m_colorSizeBytes'].append(h*w*c) # uint8 * h * w * c 
        h, w = frames['m_depthCompressed'][-1].shape
        frames['m_depthSizeBytes'].append(2*h*w) # uint16 * h * w
    
    return frames

def writeHeaderToFile(out, data):
    # out.write(m_versionNumber, sizeof(unsigned int));
    # #define M_SENSOR_DATA_VERSION 4 (line 800)
    np.array(data['m_versionNumber'], dtype=np.uint32).tofile(out)
    print('m_versionNumber', data['m_versionNumber'])
    print('len(data sensor name)', len(data['m_sensorName'])) 
    np.array([len(data['m_sensorName'])], dtype=np.uint64).tofile(out)
    print('sensor name', data['m_sensorName'].strip())
    out.write(data['m_sensorName'].encode())
    # out.write((const char*)&m_extrinsic, sizeof(mat4f));
    print(data['m_calibrationColorIntrinsic'])
    data['m_calibrationColorIntrinsic'].tofile(out)
    print(data['m_calibrationColorExtrinsic'])
    data['m_calibrationColorExtrinsic'].tofile(out)
    print(data['m_calibrationDepthIntrinsic'])
    data['m_calibrationDepthIntrinsic'].tofile(out)
    print(data['m_calibrationDepthExtrinsic'])
    data['m_calibrationDepthExtrinsic'].tofile(out)
    
#     out.write((const char*)&m_colorCompressionType, sizeof(COMPRESSION_TYPE_COLOR));
#     out.write((const char*)&m_depthCompressionType, sizeof(COMPRESSION_TYPE_DEPTH));
#     m_colorCompressionType, m_depthCompressionType = TYPE_COLOR_UNKNOWN; TYPE_COLOR_UNKNOWN = -1;
    
    # compression type is temporarily set as UNKNOWN 
    np.array([0], dtype=np.int32).tofile(out)
    np.array([0], dtype=np.int32).tofile(out)
    
    np.array(data['m_colorWidth'], dtype=np.uint32).tofile(out)
    np.array(data['m_colorHeight'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthWidth'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthHeight'], dtype=np.uint32).tofile(out)
    np.array(data['m_depthShift'], dtype=np.float32).tofile(out)
    print('depth shift', data['m_depthShift'])

def writeRGBFramesToFile_bundlefusion(out, rgbdframes):

    for i in tqdm(range(int(rgbdframes['num_frames']))):
        print(np.array(rgbdframes['m_cameraToWorld'][i], dtype=np.float32))
        np.array(rgbdframes['m_cameraToWorld'][i], dtype=np.float32).tofile(out)
        np.array(rgbdframes['m_timeStampColor'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_timeStampDepth'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_colorSizeBytes'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_depthSizeBytes'][i], dtype=np.uint64).tofile(out)
        
        rgbdframes['m_colorCompressed'][i].tofile(out)
        np.array(rgbdframes['m_depthCompressed'][i], dtype=np.uint16).tofile(out)
        #np.array(rgbdframes['m_colorCompressed'][i], dtype=np.ubyte).tofile(out)
        # if i > 100:
        #     break

def writeIMUFramesToFile(out, data):
    '''
        write the following data to the .sens file: (the order matters)
            - number of frames
            - rotationRate
            - acceleration
            - magneticField
            - attitude
            - gravity
            - timeStamp 
        default data:
            num_frames = number of rgbd images in the folder
            IMUFrame() {
                    rotationRate = vec3d(0.0);
                    acceleration = vec3d(0.0);
                    magneticField = vec3d(0.0);
                    attitude = vec3d(0.0);
                    gravity = vec3d(0.0);
                    timeStamp = 0;
                }
            
    '''
    for key in data.keys():
        print(f'writting {key} to file, value = {data[key]}')
        if 'timeStamp' in key:
            np.array(data[key], dtype=np.int64).tofile(out)
        else:
            np.array(data[key], dtype=float).tofile(out)
        
def saveToFile(data_dir, filename_out, read_frames, write_frames, dir_out='./', mode=8):
    #std::ofstream out(filename, std::ios::binary);
    if mode == 8: # SensorDataReader 
        out = open(f'{dir_out}/{filename_out}.sens', 'wb')

        header_filename = data_dir + '/info.txt'
        print('reading header...')
        data_header = read_info(header_filename)
        print('done!')
        print('reading imu file...')
        imu_filename = data_dir + '/imu.txt'
        imu_data = read_info(imu_filename)
        print('done!')
        #data_header['m_frames.size'] = 100
        imu_data['num_frames'] = data_header['m_frames.size']
        print('reading frames...')
        print('number of frames =', data_header['m_frames.size'])
        rgbdframes = read_frames(data_dir, num_frames=data_header['m_frames.size'])#read rgbframes from data dir
        print('done!')

        print('writting header...')
        writeHeaderToFile(out, data_header)
        print('done!')
        print('writting rgbd frames...')
        np.array(int(data_header['m_frames.size']), np.int64).tofile(out)

        write_frames(out, rgbdframes)
        print('done!')
        print('writting imu frames...')
        writeIMUFramesToFile(out, imu_data)
        print('done!')
    elif mode == 7: # BinaryDumpReader
        

In [32]:
saveToFile('./apt0', filename_out='apt0_new', 
           read_frames=read_frames_bundlefusion,
           write_frames=write_frames_bundlefusion,
           dir_out='C:/Users/admin/Downloads/BundleFusion-master/BundleFusion-master/FriedLiver/x64/data/')

reading header...
m_versionNumber = 4

 4
m_sensorName = StructureSensor

 StructureSensor
m_colorWidth = 640

 640
m_colorHeight = 480

 480
m_depthWidth = 640

 640
m_depthHeight = 480

 480
m_depthShift = 1000

 1000
m_calibrationColorIntrinsic = 582.871 0 320 0 0 582.871 240 0 0 0 1 0 0 0 0 1 

m_calibrationColorExtrinsic = 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 

m_calibrationDepthIntrinsic = 583 0 320 0 0 583 240 0 0 0 1 0 0 0 0 1 

m_calibrationDepthExtrinsic = 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 

m_frames.size = 8560

 8560
done!
reading imu file...
rotationRate = 0.0

 0.0
acceleration = 0.0

 0.0
magneticField = 0.0

 0.0
attitude = 0.0

 0.0
gravity = 0.0

 0.0
timeStamp = 0
 0
done!
reading frames...
number of frames = 8560
done!
writting header...
m_versionNumber 4
len(data sensor name) 15
sensor name StructureSensor
[[582.871   0.    320.      0.   ]
 [  0.    582.871 240.      0.   ]
 [  0.      0.      1.      0.   ]
 [  0.      0.      0.      1.   ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0

100%|█████████████████████████████████████████████████████████████████████████████| 8560/8560 [00:38<00:00, 222.35it/s]


done!
writting imu frames...
writting rotationRate to file, value = 0.0
writting acceleration to file, value = 0.0
writting magneticField to file, value = 0.0
writting attitude to file, value = 0.0
writting gravity to file, value = 0.0
writting timeStamp to file, value = 0
writting num_frames to file, value = 8560
done!


### Custom dataset

In [3]:
import matplotlib.pyplot as plt

In [4]:
import collections 
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])

class Image_colmap(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)
    
def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [79]:
from scipy.spatial.transform import Rotation as rot
import scipy

def read_images_text(path):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadImagesText(const std::string& path)
        void Reconstruction::WriteImagesText(const std::string& path)
    """
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image_colmap(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images

def from_colmap(images_txt):
    colmap_images = read_images_text(images_txt)
    world_to_cam = np.zeros((len(colmap_images), 4, 4))
    world_to_cam[:, 3, 3] = 1
    index = dict()
    images = dict()
    for pos_i, (i, img) in enumerate(colmap_images.items()):
        index[i] = pos_i
        images[img.name] = pos_i
        world_to_cam[pos_i, :3, 3] = img.tvec
        wxyz = img.qvec
        xyzw = np.roll(wxyz, -1)
        rot = scipy.spatial.transform.Rotation.from_quat(xyzw).as_matrix()
        world_to_cam[pos_i, :3, :3] = rot
    return world_to_cam, index, images

def unpack_float32(ar):
    r"""Unpacks an array of uint8 quadruplets back to the array of float32 values.
    Parameters
    ----------
    ar : np.naddary
        of shape [**, 4].
    Returns
    -------
    ar : np.naddary
        of shape [**]
    """
    shape = ar.shape[:-1]
    return ar.ravel().view(np.float32).reshape(shape)

def load_depthmap(file):
    r"""Loads a float32 depthmap packed into RGBA png.
    Parameters
    ----------
    file : str
    Returns
    -------
    depthmap : np.ndarray
        of shape [height, width], np.float32
    """
    depthmap = Image.open(file)
    depthmap = np.asarray(depthmap)
    depthmap = unpack_float32(depthmap)
    return depthmap

def make_matrix(data):
    return np.array([np.array(d, dtype=np.float32) for d in data[1:]]).reshape((4, 4))

def read_info(filename):
    header_data = {}
    with open(filename) as file:
        for line in file:
            print(line)
            tmp = line.strip().split('=')
            if 'Intrinsic' in tmp[0]:
                tmp_arr = np.array(tmp[-1].rstrip().split(' ')[1:], dtype=np.float32)
                intr_mat = np.eye(4, dtype=np.float32)
                intr_mat[0, 0] = tmp_arr[2]
                intr_mat[1, 1] = tmp_arr[3]
                intr_mat[0, 2] = tmp_arr[4]
                intr_mat[1, 2] = tmp_arr[5]
                print(intr_mat)
                header_data[tmp[0].rstrip()] = intr_mat
            elif 'Extrinsic' in tmp[0]:
                header_data[tmp[0].rstrip()] = make_matrix(tmp[-1].rstrip().split(' '))
            else:
                print(tmp[-1])
                header_data[tmp[0].rstrip()] = tmp[-1].strip()
                    
    return header_data

def read_frames_sk3d(folder, obj='amber_vase', light='ambient@best', num_frames=100, mode=8):
    frames = {'m_colorCompressed': [], 'm_depthCompressed': [], 'm_cameraToWorld': [],
              'm_timeStampColor': [], 'm_timeStampDepth': [], 
              'm_colorSizeBytes': [], 'm_depthSizeBytes': []}
    frames['num_frames'] = num_frames
    folder_frames = folder + '/processed_scans/images/'
    for i in range(int(num_frames)):
        frames['m_colorCompressed'].append(read_image(folder_frames + f'undist/{obj}/tis_right/rgb/{light}/{i:04d}.png'))
        frames['m_timeStampColor'].append(i)
        d = load_depthmap(folder_frames + f'depthmaps/{obj}/stl@tis_right.undist/{i:04d}.png')
#         d[np.isnan(d)] = 0
        if mode == 8:
            d = (d*65535).astype(np.uint16)
        
        frames['m_depthCompressed'].append(d)
        
        frames['m_timeStampDepth'].append(i)
        h, w, c = frames['m_colorCompressed'][-1].shape
        frames['m_colorSizeBytes'].append(h*w*c) # uint8 * h * w * c 
        h, w = frames['m_depthCompressed'][-1].shape
        frames['m_depthSizeBytes'].append(2*h*w) # uint16 * h * w
      
    world_to_cam, index, images = from_colmap(folder + '/calibration/undist_extrinsics/_calibration/tis_right.images.txt')
    frames['m_cameraToWorld'] = [np.linalg.inv(mat) for mat in world_to_cam]
    print('m_cameraToWorld', frames['m_cameraToWorld'])

            
    return frames

def writeRGBFramesToFile_sk3d(out, rgbdframes):

    for i in tqdm(range(int(rgbdframes['num_frames']))):
        np.array(rgbdframes['m_cameraToWorld'][i], dtype=np.float32).tofile(out)
        np.array(rgbdframes['m_timeStampColor'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_timeStampDepth'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_colorSizeBytes'][i], dtype=np.uint64).tofile(out)
        np.array(rgbdframes['m_depthSizeBytes'][i], dtype=np.uint64).tofile(out)
        
        rgbdframes['m_colorCompressed'][i].tofile(out)
        rgbdframes['m_depthCompressed'][i].tofile(out)
        #np.array(rgbdframes['m_colorCompressed'][i], dtype=np.ubyte).tofile(out)
        # if i > 100:
        #     break

def read_info():
    info = {}
    
    return info
        
def writeBinaryDumpFile(out, rgbdframes, info):
    out.write(info['m_sensorName'].encode())
    np.array(int(info['m_DepthNumFrames']), np.uintc).tofile(out)
    np.array(info['m_DepthImageWidth'], dtype=np.uintc).tofile(out)
    np.array(info['m_DepthImageHeight'], dtype=np.uintc).tofile(out)
    
    np.array(int(info['m_ColorNumFrames']), np.uintc).tofile(out)
    np.array(info['m_ColorImageWidth'], dtype=np.uintc).tofile(out)
    np.array(info['m_ColorImageHeight'], dtype=np.uintc).tofile(out)
    
    info['m_calibrationColorIntrinsic'].tofile(out)
    info['m_calibrationColorIntrinsicInverse'].tofile(out)
    info['m_calibrationColorExtrinsic'].tofile(out)
    info['m_calibrationColorExtrinsicInverse'].tofile(out)
    
    info['m_calibrationDepthIntrinsic'].tofile(out)
    info['m_calibrationDepthIntrinsicInverse'].tofile(out)
    info['m_calibrationDepthExtrinsic'].tofile(out)
    info['m_calibrationDepthExtrinsicInverse'].tofile(out)
    
    for i in range(int(info['m_DepthNumFrames'])):
        rgbdframes['m_depthCompressed'][i][i].resize(info['m_DepthImageWidth']*info['m_DepthImageHeight']).tofile(out)
    for i in range(int(info['m_DepthNumFrames'])):
        ones = np.ones((info['m_DepthImageWidth']*info['m_DepthImageHeight'], 1))
        rgbw_image = np.cat((rgbdframes['m_colorCompressed'][i].resize(info['m_DepthImageWidth']*info['m_DepthImageHeight'], 3), ones), axis=1)
        np.array(rgbw_image, dtype=np.uintc).tofile(out)
    
    np.arange(0, int(info['m_DepthNumFrames']), dtype=np.uint64).tofile(out) # timestamps  
    np.arange(0, int(info['m_DepthNumFrames']), dtype=np.uint64).tofile(out) # timestamps 
    np.array(rgbdframes['m_cameraToWorld'], dtype=np.float32).tofile(out)
    
# BINARY DUMP READER        
# s >> sensorData.m_SensorName;
# s >> sensorData.m_DepthNumFrames;
# s >> sensorData.m_DepthImageWidth;
# s >> sensorData.m_DepthImageHeight;
# s >> sensorData.m_ColorNumFrames;
# s >> sensorData.m_ColorImageWidth;
# s >> sensorData.m_ColorImageHeight;
# s >> sensorData.m_CalibrationDepth;
# s >> sensorData.m_CalibrationColor;

# sensorData.m_DepthImages.resize(sensorData.m_DepthNumFrames);
# sensorData.m_ColorImages.resize(sensorData.m_ColorNumFrames);

# for (size_t i = 0; i < sensorData.m_DepthImages.size(); i++) {
#     sensorData.m_DepthImages[i] = new float[sensorData.m_DepthImageWidth*sensorData.m_DepthImageHeight];
#     s.readData((BYTE*)sensorData.m_DepthImages[i], sizeof(float)*sensorData.m_DepthImageWidth*sensorData.m_DepthImageHeight);
# }
# for (size_t i = 0; i < sensorData.m_ColorImages.size(); i++) {
#     sensorData.m_ColorImages[i] = new vec4uc[sensorData.m_ColorImageWidth*sensorData.m_ColorImageHeight];
#     s.readData((BYTE*)sensorData.m_ColorImages[i], sizeof(vec4uc)*sensorData.m_ColorImageWidth*sensorData.m_ColorImageHeight);
# }

# s >> sensorData.m_ColorImagesTimeStamps;
# s >> sensorData.m_DepthImagesTimeStamps;

# s >> sensorData.m_trajectory;

In [80]:
saveToFile(data_dir='./sk3d_v0_ambervase/', 
           filename_out='sk3d_ambervase_gt_traj', 
           read_frames=read_frames_sk3d,
           write_frames=writeRGBFramesToFile_sk3d,
           dir_out='C:/Users/admin/Downloads/BundleFusion-master/BundleFusion-master/FriedLiver/x64/data/')

reading header...
m_versionNumber = 4

 4
m_sensorName = StructureSensor

 StructureSensor
m_colorWidth = 2357

 2357
m_colorHeight = 1958

 1958
m_depthWidth = 2357

 2357
m_depthHeight = 1958

 1958
m_depthShift = 65535

 65535
m_calibrationColorIntrinsic = 2357 1958 2291.7852039710788 2294.9830320504343 1177.495158214813 975.6468065598369

[[2.2917852e+03 0.0000000e+00 1.1774951e+03 0.0000000e+00]
 [0.0000000e+00 2.2949829e+03 9.7564679e+02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00]]
m_calibrationColorExtrinsic = 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1

m_calibrationDepthIntrinsic = 2357 1958 2291.7852039710788 2294.9830320504343 1177.495158214813 975.6468065598369

[[2.2917852e+03 0.0000000e+00 1.1774951e+03 0.0000000e+00]
 [0.0000000e+00 2.2949829e+03 9.7564679e+02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00]]
m_c

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 126.12it/s]


done!
writting imu frames...
writting rotationRate to file, value = 0.0
writting acceleration to file, value = 0.0
writting magneticField to file, value = 0.0
writting attitude to file, value = 0.0
writting gravity to file, value = 0.0
writting timeStamp to file, value = 0
writting num_frames to file, value = 100
done!
